In [3]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os,sys

dirpath = os.getcwd().split("code")[0]+"code/"
sys.path.append(dirpath)
from pre_process import clean_LC

folder = "../../KOI_Data/"

In [ ]:
##probar con 10 curvas...

### Read light curves
> From Kepler or all others

In [4]:
T = 70000

lc_train = np.random.rand(4000,T)-1
lc_train.shape

(4000, 70000)

### Pre-process (clean lc)
> And save it

In [5]:
process_lc = []
for i in range(lc_train.shape[0]):
    lc_cleaned = clean_LC(lc_train[i], kernel_median=25, kernel_pol=151, detrend_median=False, plot=False) 
    process_lc.append(lc_cleaned)
lc_train = np.asarray(process_lc)
lc_train.shape

Clean done (remove outliers iterativetly), erase 0 values
Clean done (remove outliers iterativetly), erase 0 values
Clean done (remove outliers iterativetly), erase 0 values
Clean done (remove outliers iterativetly), erase 0 values
Clean done (remove outliers iterativetly), erase 0 values
Clean done (remove outliers iterativetly), erase 0 values
Clean done (remove outliers iterativetly), erase 0 values
Clean done (remove outliers iterativetly), erase 0 values
Clean done (remove outliers iterativetly), erase 0 values
Clean done (remove outliers iterativetly), erase 0 values
Clean done (remove outliers iterativetly), erase 0 values
Clean done (remove outliers iterativetly), erase 0 values
Clean done (remove outliers iterativetly), erase 0 values
Clean done (remove outliers iterativetly), erase 0 values
Clean done (remove outliers iterativetly), erase 0 values
Clean done (remove outliers iterativetly), erase 0 values
Clean done (remove outliers iterativetly), erase 0 values
Clean done (re

Clean done (remove outliers iterativetly), erase 0 values
Clean done (remove outliers iterativetly), erase 0 values
Clean done (remove outliers iterativetly), erase 0 values
Clean done (remove outliers iterativetly), erase 0 values
Clean done (remove outliers iterativetly), erase 0 values
Clean done (remove outliers iterativetly), erase 0 values
Clean done (remove outliers iterativetly), erase 0 values
Clean done (remove outliers iterativetly), erase 0 values


KeyboardInterrupt: 

In [ ]:
## maybe save the processed light curve
np.save('lc_train_processed.npy', lc_train)

In [ ]:
lc_train = np.load('lc_train_processed.npy')

### Generate representation
> Continous values (with zeros)  
> Coupled values (adding time)

In [ ]:
#do it..

### Process to neural network 
> center?  
> With what value scale? (min, iqr, std)

In [ ]:
...

### Define data augmentation
> description..

In [ ]:
def noisify_samples(inputs, outputs, errors, batch_size=500, sample_weight=None):
    """
    inputs: {'main_input': X, 'aux_input': X[:, :, [1]]}
    outputs: X[:, :, [1]]
    errors: X_raw[:, :, 2]
    """
    if sample_weight is None:
        sample_weight = np.ones(errors.shape)
    X = inputs['main_input']
    X_aux = inputs['aux_input']
    shuffle_inds = np.arange(len(X))
    while True:
        # New epoch
        np.random.shuffle(shuffle_inds)
        noise = errors * np.random.normal(size=errors.shape)
        X_noisy = X.copy()
        X_noisy[:, :, 1] += noise
        # Re-scale to have mean 0 and std dev 1; TODO make this optional
        X_noisy[:, :, 1] -= np.atleast_2d(np.nanmean(X_noisy[:, :, 1], axis=1)).T
        X_noisy[:, :, 1] /= np.atleast_2d(np.std(X[:, :, 1], axis=1)).T

        for i in range(ceil(len(X) / batch_size)):
            inds = shuffle_inds[(i * batch_size):((i + 1) * batch_size)]
            yield ([X_noisy[inds], X_aux[inds]], X_noisy[inds, :, 1:2], sample_weight[inds])
            
history = model.fit_generator(noisify_samples(X, Y, errors, batch_size,sample_weight),
                              samples_per_epoch=len(Y), nb_epoch=nb_epoch, validation_data=validation_data)


from keras.utils.data_utils import Sequence
class KOIGenerator(Sequence):
    def __init__(self, x_set, batch_size, flip_p=True, add_n = True, shift=True ):
        self.x = x_set
        self.batch_size = batch_size
        self.flip_p = flip_p
        self.add_n = add_n
        self.shift = shift
        
    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))
    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        return np.stack([self.augment(x) for x in batch_x], axis=0)
            
    def augment(self,x):
        """Do Augmentation"""
        return_x = x.copy()[:,0]
        T = int(return_x.shape[0])
        
        if self.flip_p: #np.random.rand() <= self.flip_p: 
            if np.random.rand() <= 0.5:
                print("Flip")
                return_x = return_x[::-1] #flip
        
        if self.add_n: #np.random.rand() <= self.add_n: 
            if np.random.rand() <= 0.5:
                print("Noise added")
                noise_level =  np.random.uniform(0.01,0.45)
                noise_mask = np.random.binomial(n=1, p=noise_level, size=return_x.shape)
                return_x = return_x[:]*noise_mask
                
        if self.shift:
            if np.random.rand() <= 0.5:
                print("Shift")
                v_shift = np.random.randint(1, int(T*0.49))  #valor a shift?
                return_x = return_x[v_shift:]
                #padd
                return_x = keras.preprocessing.sequence.pad_sequences([return_x], maxlen=T, 
                                    padding='pre', value =0, dtype='float32')[0]
        return return_x[:,None]
                    
# Training data
train_gen = KOIGenerator(X_train, 128)

# Validation data
valid_gen = KOIGenerator(X_test, 128, flip_p = False, add_n = False, shift=False)


# Training
autoencoder.fit_generator(
    train_gen,
    epochs=20,
    validation_data=valid_gen,
    workers=10, use_multiprocessing=True)

In [ ]:
## define model

In [ ]:
## train model (1era fase)

In [ ]:
### evaluate with loss and measure acc on labeled data

In [ ]:
### load metadata

In [ ]:
#pre-process: scale metadata

In [ ]:
### train model (2da fase)

In [ ]:
### evaluate with mse between metadata (and the values before)